##### Importing Libraries

In [4]:
import pandas as pd
from ipywidgets import widgets

##### Input File name

In [5]:
#As the script need to be used for all states, entering file name in the text box calls the data file dynamically

text=widgets.Text()
display("Enter the State name")
display(text)

def handle_submit(sender):
    print(text.value)

text.on_submit(handle_submit)

'Enter the State name'

Text(value='')

Goa


In [6]:
state=text.value
print(state)

Goa


##### Reading Excel file

In [7]:
xls = pd.ExcelFile("../Population_Dispersion/"+state+".xlsx")
xls.sheet_names

['gender', 'age dispersion']

In [8]:
age_dispersion = pd.read_excel(state+".xlsx", sheet_name="age dispersion")
age_dispersion.shape

(204, 8)

In [9]:
state_data=pd.read_excel(state+".xlsx", sheet_name="gender", encoding="latin-1")
state_data.shape

(697, 11)

##### Data Cleaning

In [10]:
#Filtering data for "Ward" and "Village"
#Renaming columns in both "gender" and "age dispersion" excel sheet
#Removing extra spaces in the name column
#Removing rows with value "0" for "No of Ind i.e e records"/"E_records"

state_data=state_data[(state_data["Level"]=="VILLAGE") | (state_data["Level"]=="WARD")]
state_data=state_data.rename(columns={"% Male":"Male_Percent", "% Female":"Female_Percent", "No of Ind i.e e records":"E_records"})
age_dispersion=age_dispersion.rename(columns={"Distt.":"District", "M-Rural":"Male-Rural", "F-Rural":"Female-Rural", "M-Urban":"Male-Urban", "F-Urban":"Female-Urban"})
state_data["Name"]=state_data["Name"].str.strip()
state_data = state_data[state_data.E_records != 0]


In [11]:
state_data.columns

Index(['State', 'District', 'Subdistt', 'Town/Village', 'Ward', 'Level',
       'Name', 'TRU', 'E_records', 'Male_Percent', 'Female_Percent'],
      dtype='object')

In [12]:
state_data.shape

(573, 11)

In [13]:
age_dispersion.columns

Index(['State', 'District', 'Area Name', 'Age', 'Male-Rural', 'Female-Rural',
       'Male-Urban', 'Female-Urban'],
      dtype='object')

##### Calculating population based on gender

In [14]:
#Adding a column gender with value "Male" and "Female" for each records in the data.

gender={'Gender':['Male','Female']}
gender_frame=pd.DataFrame(gender)
state_data['tmp']=1
gender_frame['tmp']=1
state_data=pd.merge(state_data,gender_frame, on=['tmp'])
state_data=state_data.drop(columns='tmp')

In [15]:
state_data

,State,District,Subdistt,Town/Village,Ward,Level,Name,TRU,E_records,Male_Percent,Female_Percent,Gender
0,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male
1,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Female
2,30,585,5609,626636,0,VILLAGE,Querim,Rural,3311.42,52.369980,47.630020,Male
3,30,585,5609,626636,0,VILLAGE,Querim,Rural,3311.42,52.369980,47.630020,Female
4,30,585,5609,626637,0,VILLAGE,Paliem,Rural,3025.84,52.089337,47.910663,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
1141,30,586,5619,803254,8,WARD,Canacona (M Cl) WARD NO.-0008,Urban,1153.90,53.098189,46.901811,Female
1142,30,586,5619,803254,9,WARD,Canacona (M Cl) WARD NO.-0009,Urban,1312.30,50.963956,49.036044,Male
1143,30,586,5619,803254,9,WARD,Canacona (M Cl) WARD NO.-0009,Urban,1312.30,50.963956,49.036044,Female
1144,30,586,5619,803254,10,WARD,Canacona (M Cl) WARD NO.-0010,Urban,1537.80,50.143062,49.856938,Male


In [16]:
state_data.shape

(1146, 12)

In [17]:
state_data.head(10)

,State,District,Subdistt,Town/Village,Ward,Level,Name,TRU,E_records,Male_Percent,Female_Percent,Gender
0,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male
1,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Female
2,30,585,5609,626636,0,VILLAGE,Querim,Rural,3311.42,52.369980,47.630020,Male
3,30,585,5609,626636,0,VILLAGE,Querim,Rural,3311.42,52.369980,47.630020,Female
4,30,585,5609,626637,0,VILLAGE,Paliem,Rural,3025.84,52.089337,47.910663,Male
5,30,585,5609,626637,0,VILLAGE,Paliem,Rural,3025.84,52.089337,47.910663,Female
6,30,585,5609,626638,0,VILLAGE,Corgao,Rural,7236.51,51.046844,48.953156,Male
7,30,585,5609,626638,0,VILLAGE,Corgao,Rural,7236.51,51.046844,48.953156,Female
8,30,585,5609,626639,0,VILLAGE,Pernem,Rural,2944.09,51.462421,48.537579,Male
9,30,585,5609,626639,0,VILLAGE,Pernem,Rural,2944.09,51.462421,48.537579,Female


In [18]:
#Calculating "Male" and "Female" population in each village/ward
gender=["Male", "Female"]
for i in gender:
    state_data.loc[state_data["Gender"]==i, "Gender_Pop"]=state_data['E_records']*(state_data[i+"_Percent"]/100)    

In [19]:
state_data.head()

,State,District,Subdistt,Town/Village,Ward,Level,Name,TRU,E_records,Male_Percent,Female_Percent,Gender,Gender_Pop
0,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35
1,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Female,98.10
2,30,585,5609,626636,0,VILLAGE,Querim,Rural,3311.42,52.369980,47.630020,Male,1734.19
3,30,585,5609,626636,0,VILLAGE,Querim,Rural,3311.42,52.369980,47.630020,Female,1577.23
4,30,585,5609,626637,0,VILLAGE,Paliem,Rural,3025.84,52.089337,47.910663,Male,1576.14


##### Calculating population dispersion

In [20]:
#Merging both "gender" and "age dispersion" data frames based on state and district code
pop_dispersion=pd.merge(state_data,age_dispersion, on=["State","District"])
pop_dispersion.shape

(116892, 19)

In [21]:
pop_dispersion.head(5)

,State,District,Subdistt,Town/Village,Ward,Level,Name,TRU,E_records,Male_Percent,Female_Percent,Gender,Gender_Pop,Area Name,Age,Male-Rural,Female-Rural,Male-Urban,Female-Urban
0,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),0,1.350148,1.178122,1.234329,1.265854
1,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),1,1.438612,1.397351,1.368443,1.414411
2,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),2,1.364180,1.284942,1.392971,1.323280
3,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),3,1.306831,1.290531,1.318595,1.326609
4,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),4,1.344047,1.302331,1.378729,1.361147


In [22]:
TRU=["Rural", "Urban"]
gender=["Male", "Female"]

for i in TRU:
    for j in gender:
        value=j+"-"+i
        print(value)
        pop_dispersion.loc[(pop_dispersion["TRU"]==i) & ( pop_dispersion["Gender"]==j), "Population"]=pop_dispersion["Gender_Pop"]*(pop_dispersion[j+"-"+i]/100)
        

Male-Rural
Female-Rural
Male-Urban
Female-Urban


In [23]:
pop_dispersion.head(5)

,State,District,Subdistt,Town/Village,Ward,Level,Name,TRU,E_records,Male_Percent,Female_Percent,Gender,Gender_Pop,Area Name,Age,Male-Rural,Female-Rural,Male-Urban,Female-Urban,Population
0,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),0,1.350148,1.178122,1.234329,1.265854,1.692410
1,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),1,1.438612,1.397351,1.368443,1.414411,1.803300
2,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),2,1.364180,1.284942,1.392971,1.323280,1.710000
3,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),3,1.306831,1.290531,1.318595,1.326609,1.638112
4,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,223.45,56.097561,43.902439,Male,125.35,District - North Goa (01),4,1.344047,1.302331,1.378729,1.361147,1.684762


In [24]:
#Removing unwanted columns and changing column position
pop_dispersion=pop_dispersion.drop(columns=["E_records","Male_Percent", "Female_Percent", "Gender_Pop", "Area Name", "Male-Rural", "Female-Rural", "Male-Urban", "Female-Urban"])
pop_dispersion = pop_dispersion[["State", "District", "Subdistt", "Town/Village", "Ward", "Level", "Name", "TRU",  "Age", "Gender", "Population"]]

In [25]:
pop_dispersion.head()

,State,District,Subdistt,Town/Village,Ward,Level,Name,TRU,Age,Gender,Population
0,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,0,Male,1.692410
1,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,1,Male,1.803300
2,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,2,Male,1.710000
3,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,3,Male,1.638112
4,30,585,5609,626635,0,VILLAGE,Tiracol,Rural,4,Male,1.684762


In [26]:
pop_dispersion.shape

(116892, 11)

##### Writing the output to csv file

In [27]:
writer=pop_dispersion.to_csv(r"../Population_Dispersion/"+state+"_Output.csv")

##### Verification

In [28]:
Pop_test=pd.DataFrame(pop_dispersion.groupby(["State", "District"]).agg({"Population":"sum"})).reset_index()
Pop_test

,State,District,Population
0,30,585,891628.72
1,30,586,704590.70


In [29]:
Pop_test=pd.DataFrame(pop_dispersion.groupby(["State", "District","Town/Village", "Name"]).agg({"Population":"sum"})).reset_index()
Pop_test.head(20)

,State,District,Town/Village,Name,Population
0,30,585,626635,Tiracol,223.45
1,30,585,626636,Querim,3311.42
2,30,585,626637,Paliem,3025.84
3,30,585,626638,Corgao,7236.51
4,30,585,626639,Pernem,2944.09
5,30,585,626640,Poroscodem,735.75
6,30,585,626641,Casnem,347.71
7,30,585,626642,Varconda,2406.72
8,30,585,626643,Amberem,371.69
9,30,585,626644,Uguem,1234.97
